In [45]:
import torch

In [51]:
r = torch.randn(4, 3)
r2 = torch.randn(4, 3)

In [52]:
r

tensor([[ 0.8244,  0.3914, -0.8777],
        [ 0.4770, -0.6141, -0.5484],
        [ 0.3074,  0.3883, -1.1045],
        [-0.8934,  1.3041, -1.2335]])

In [53]:
import torch.nn.functional as F

In [54]:
F.normalize(r, p=2, dim=-1) 

tensor([[ 0.6511,  0.3091, -0.6932],
        [ 0.5013, -0.6454, -0.5763],
        [ 0.2539,  0.3208, -0.9125],
        [-0.4456,  0.6504, -0.6152]])

In [56]:
r2

tensor([[-1.0819, -1.6316, -0.4554],
        [ 0.7388, -0.8986, -0.0307],
        [-1.5274,  0.0625, -1.7072],
        [ 0.8688,  2.1102,  1.1983]])

In [62]:
(r[0,0] - r2[0,0])**2

tensor(3.6338)

In [ ]:
F.mse_loss(r, r2, reduction='sum')

tensor(21.8268)

: 

: 

In [66]:
F.mse_loss(r, r2, reduction='none').mean(dim=1)

tensor([2.6349, 0.1392, 1.2786, 3.2229])

In [67]:
F.mse_loss(r, r2, reduction='none').mean()

tensor(1.8189)

In [68]:
F.mse_loss(r, r2, reduction='none').mean(dim=1).sum()

tensor(7.2756)

In [4]:
import time
# time in seconds
start = time.time()
for i in range(1000000):
    pass
end = time.time()
print("Time taken for 1 million iterations: ", end - start)

Time taken for 1 million iterations:  0.027349233627319336
